<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homeworks 8 & 9 (Combined): Practical - Faces <br> <br>
<font color=686880 size=4>
    TAs: Arman Malekzadeh - Amirhossein Ramazani Bonab - Yalda Shabanzadeh
    
____

### Full Name : Erfan Sadraiye
### Student Number : 99101835
### Colab Link: https://colab.research.google.com/drive/1SI8RoUyPxsPoqgYx4-W8vs26M5VG4Hr_?usp=sharing
___

In this assignment, you'll be dealing with a dataset consisting of 400 faces, belonging to 40 people. Your main task is to train models capable of recognizing those faces.
The faces are `jpg` images packed together as a zip file called `images.zip`, and the `meta.csv` determines from which person each image was taken.

In [ ]:
!wget -O images.zip https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
!wget -O meta.csv https://www.dropbox.com/s/vqt1v8sfmz18rcf/faces-metadata.csv?dl=1

--2023-06-24 12:03:29--  https://www.dropbox.com/s/wrias0bjyte3rr2/images.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/wrias0bjyte3rr2/images.zip [following]
--2023-06-24 12:03:29--  https://www.dropbox.com/s/dl/wrias0bjyte3rr2/images.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc720dbadd7cdb0011f2c789728d.dl.dropboxusercontent.com/cd/0/get/B-kXR9Y4sEqQber5MYuZpRO7bxcGQ-WDoRmaK4N_plk6M05Af-LQaDpjHZ8tvy1og5ONdGesrohEFsKq-Nwmk_n2OTVsvLAXvoWjb2epKd9n3cqVY40FVOEm0cG6nK0h5cMtKs3-DS--N9-r3QiYc8lyZ9i7O4DJWQiG1Pewe4v1whk5WrBsTFz3s9nc4Be2UUY/file?dl=1# [following]
--2023-06-24 12:03:30--  https://uc720dbadd7cdb0011f2c789728d.dl.dropboxusercontent.com/cd/0/get/B-kXR9Y4sEqQber5MYuZpRO7bxcGQ-WDoRmaK4N_plk6M05Af-LQaDpjHZ8t

In [ ]:
!unzip images.zip -d /content/images

Archive:  images.zip
   creating: /content/images/content/img-data/
 extracting: /content/images/content/img-data/50.jpg  
 extracting: /content/images/content/img-data/181.jpg  
 extracting: /content/images/content/img-data/126.jpg  
 extracting: /content/images/content/img-data/254.jpg  
 extracting: /content/images/content/img-data/249.jpg  
 extracting: /content/images/content/img-data/164.jpg  
 extracting: /content/images/content/img-data/275.jpg  
 extracting: /content/images/content/img-data/204.jpg  
 extracting: /content/images/content/img-data/203.jpg  
 extracting: /content/images/content/img-data/346.jpg  
 extracting: /content/images/content/img-data/165.jpg  
 extracting: /content/images/content/img-data/318.jpg  
 extracting: /content/images/content/img-data/95.jpg  
 extracting: /content/images/content/img-data/92.jpg  
 extracting: /content/images/content/img-data/87.jpg  
 extracting: /content/images/content/img-data/127.jpg  
 extracting: /content/images/content/img

In [ ]:
# you can use the following packages
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly.express as px

from sklearn.svm import SVC
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score, f1_score
from cvxopt import matrix, solvers

# Sec 1: Data Preparation

Load all images as a numpy array (called `x`) and the metadata as a dataframe **(P1-1: 25 points)**

In [ ]:
df = pd.read_csv('meta.csv')
x = np.array([np.array(Image.open('/content/images/content/img-data/' + img)) for img in df['path']])

Flatten and normalize the images **(P1-2: 10 points)**

Note: For this part, normalization only means dividing by 255.

In [ ]:
x = x.reshape(x.shape[0], -1)
x = x / 255

Encode the labels using `LabelEncoder` and save the result as a numpy array called `y` **(P1-3: 5 points)**

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

Split the data into training (80%), validation (10%), and testing (10%) parts. **(P1-4: 10 points)**

Note: The split procedure should maintain the class balance. This is sometimes called "[stratified sampling](https://en.wikipedia.org/wiki/Stratified_sampling)".

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y , random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# Sec 2: SVM

## Subsec 2.1.

Train a SVM using `sklearn`. Note that the hyperparameter tuning should be done using the "validation" set. **(P2-1-1: 20 points)**

In [ ]:
svc_parameters = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': [0.1, 0.01, 0.001]
}

best_accuracy = 0.0
best_svm = None
best_C = None
best_kernel = None
best_gamma = None

for C in svc_parameters['C']:
    for kernel in svc_parameters['kernel']:
        for gamma in svc_parameters['gamma']:
            svm = SVC(C=C, kernel=kernel, gamma=gamma)
            svm.fit(x_train, y_train)
            val_accuracy = svm.score(x_val, y_val)
            if val_accuracy > best_accuracy:
                best_C = C
                best_kernel = kernel
                best_gamma = gamma
                best_accuracy = val_accuracy
                best_svm = svm
print('Best C:', best_C)
print('Best kernel:', best_kernel)
print('Best gamma:', best_gamma)
print('Best accuracy:', best_accuracy)

Best C: 0.1
Best kernel: linear
Best gamma: 0.1
Best accuracy: 0.975


Report the accuracy and F1-score of the classifier on the testing set **(P2-1-2: 10 points)**

In [ ]:
y_pred_svm = best_svm.predict(x_test)
print('Accuracy:', accuracy_score(y_test, y_pred_svm))
print('F1-score:', f1_score(y_test, y_pred_svm, average='macro'))

Accuracy: 0.975
F1-score: 0.9829059829059827


## Subsec 2.2.

Train a SVM from scratch using the following class **(P2-2-1: 40 points)**

Note: For this part, you are not required to use the validation set for hyperparameter tuning. However, you might get better results if you do that.

In [ ]:
class SVM:
    def __init__(self, tol=1e-5):
        self.w = None
        self.b = None
        self.classes = None
        self.tol = tol

    def fit(self, X, y):
        """Fit the SVM model to dataset X in one-vs-one manner.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        y : array-like of shape (n_samples,)
            Training labels, where `n_samples` is the number of samples.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        n_samples, n_features = X.shape
        y = y.reshape(-1, 1)
        self.classes = np.unique(y)
        classes = self.classes
        n_class = len(classes)
        n_weights = n_class * (n_class - 1) // 2
        self.w = np.zeros((n_weights, n_features))
        self.b = np.zeros((n_weights, 1))
        count = 0
        for i in range(n_class):
            for j in range(i + 1, n_class):
                X_ij = X[np.logical_or(y == classes[i], y == classes[j]).reshape(-1)]
                y_ij = y[np.logical_or(y == classes[i], y == classes[j]).reshape(-1)]
                y_ij = np.where(y_ij == classes[i], 1, -1)
                w_ij, b_ij = self.fit_2class(X_ij, y_ij)
                self.w[count] = w_ij.reshape(-1)
                self.b[count] = b_ij
                count += 1
        return self

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        y_pred : ndarray of shape (n_samples)
            Your predictions for the class of each sample
        """
        n_samples, n_features = X.shape
        predicted_y = np.dot(X, self.w.T) + self.b.T
        n_class = len(self.classes)
        predicted_y = np.sign(predicted_y)
        y_pred = np.zeros((n_samples, n_class))
        count = 0
        for i in range(n_class):
            for j in range(i + 1, n_class):
                y_pred[:, i] += np.where(predicted_y[:, count] == 1, 1, 0)
                y_pred[:, j] += np.where(predicted_y[:, count] == -1, 1, 0)
                count += 1
        y_pred = np.argmax(y_pred, axis=1)
        y_pred = np.array([self.classes[i] for i in y_pred])
        return y_pred

    def fit_2class(self, X, y):
        n_samples, n_features = X.shape
        # 1/2 alpha^T P alpha + q^T alpha, G alpha <= h, A alpha = b
        kernel = np.dot(X, X.T) * np.dot(y, y.T)
        P = kernel
        q = -np.ones(n_samples).reshape(-1, 1)
        G = -np.eye(n_samples)
        h = np.zeros(n_samples).reshape(-1, 1)
        A = y.reshape(1, -1)
        b = 0

        P = matrix(P, tc='d')
        q = matrix(q)
        G = matrix(G)
        h = matrix(h)
        A = matrix(A, tc='d')
        b = matrix(b, tc='d')
        solvers.options['show_progress'] = False
        solvers.options['abstol'] = 1e-10
        solvers.options['reltol'] = 1e-10
        solvers.options['feastol'] = 1e-10

        sol = solvers.qp(P, q, G, h, A, b)
        self.alphas = np.array(sol['x'])
        support_vectors_idx = np.where(self.alphas > self.tol)[0]
        support_vectors = X[support_vectors_idx]
        y_support_vectors = y[support_vectors_idx]
        w = np.zeros(n_features)
        b = 0
        if len(support_vectors_idx) != 0:
            w = np.dot(X.T, self.alphas * y)
            b = np.mean(y_support_vectors - np.dot(support_vectors, w.reshape(-1, 1)))
        return w, b


Report the accuracy and F1-score of the classifier on the testing set **(P2-2-2: 10 points)**

In [ ]:
svm = SVM()
svm.fit(x_train, y_train.reshape(-1, 1))
y_pred_svm = svm.transform(x_test)
print('Accuracy:', accuracy_score(y_test, y_pred_svm))
print('F1-score:', f1_score(y_test, y_pred_svm, average='macro'))

Accuracy: 0.975
F1-score: 0.9829059829059827


# Sec 3: PCA

Fetch all images taken from `jones`, `taylor`, `anderson` and `wilson` (use the `labelencoder` object). Perform PCA from scratch using the following `class` to reduce the dimension of those images to 3. Save the results as a `numpy` array of shape `(40, 3)`. **(P3-1: 30 points)**

In [ ]:
class PCA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None

    def fit(self, X):
        """Fit the model with X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        self : object
            Returns the instance itself.
        """
        X -= X.mean(axis=0)
        U, S, Vt = np.linalg.svd(X)
        V = Vt.T
        self.components_ = V[:, :self.n_components]
        return self

    def transform(self, X):
        """Fit the model with X and apply the dimensionality reduction on X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        Returns
        -------
        X_new : ndarray of shape (n_samples, n_components)
            Transformed values.
        """
        assert self.components_ is not None
        X -= X.mean(axis=0)
        return np.dot(X, self.components_)


In [ ]:
persons = ['jones', 'taylor', 'anderson', 'wilson']
persons_idx = [label_encoder.transform([person])[0] for person in persons]
y_4person = y[np.isin(y, persons_idx)]
x_4person = x[np.isin(y, persons_idx)]

In [ ]:
pca = PCA(n_components=3)
pca.fit(x_4person)
transformed_x = pca.transform(x_4person)

Use `plotly` to plot these 3D samples. You should use their labels as the determiner of their color. **(P3-2: 10 points)**

In [ ]:
fig = px.scatter_3d(
    transformed_x,
    x=0,
    y=1,
    z=2,
    color=y_4person,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    opacity=0.7,
    title='PCA',
)
fig.show()

# Sec 4: K-Means

Using K-means, we want to perform clustering on the 3D samples. Consider 4 clusters. After running the K-means algorithm on the data, determine to which cluster each sample belongs. Then, use `plotly` to plot the 3D samples colored with respect to their clusters. **(P4-1: 15 points)**

In [ ]:
class KMeans:
    """
    Fits it on data, then uses predict to get cluster labels.
    """

    def __init__(self, num_clusters):
        self.num_clusters = num_clusters
        self.cluster_centers_ = None
        self.labels_ = None

    def fit(self, x, y):
        """Fits the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = x.shape
        self.cluster_centers_ = x[np.random.choice(n_samples, self.num_clusters, replace=False)]
        self.labels_ = np.zeros(n_samples)

        converged = False
        while not converged:
            self.labels_ = self._find_new_clusters(x)
            new_centers = self._compute_centers(x)
            if np.allclose(self.cluster_centers_, new_centers):
                converged = True
            self.cluster_centers_ = new_centers

        return self

    def predict(self, x):
        """ Predicts the cluster label
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted cluster label
        """

        y_pred = self._find_new_clusters(x)
        return y_pred

    def _find_new_clusters(self, x):
        centers = self.cluster_centers_
        dists = np.zeros((x.shape[0], centers.shape[0]))
        for i in range(centers.shape[0]):
            dists[:, i] = np.linalg.norm(x - centers[i], axis=1)
        return np.argmin(dists, axis=1)

    def _compute_centers(self, x):
        centers = np.zeros((self.num_clusters, x.shape[1]))
        for i in range(self.num_clusters):
            centers[i] = np.mean(x[self.labels_ == i], axis=0)
        return centers

In [ ]:
kmeans = KMeans(num_clusters=4)
kmeans.fit(transformed_x, y_4person)
y_pred_kmeans = kmeans.predict(x_4person)

In [ ]:
fig = px.scatter_3d(
    x_4person,
    x=0,
    y=1,
    z=2,
    color=y_pred_kmeans,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    opacity=0.7,
    title='K-Means',
)
fig.show()

Measure the quality of the clustering using "silhoutte score" **(P4-2: 5 points)**

Note: You can use `sklearn` or other packages to calculate the score.

In [ ]:
print(f'Silhouette score: {silhouette_score(x_4person, y_4person)}')

Silhouette score: 0.11785093591517164


Compare the result of the clustering with the original labels. To be more precise, in this part, you should clearly answer the following questions with details:<br>

- Does each cluster represent a true label?
- What is your conclusion?

**(P4-3: 10 points)**

In [ ]:
print(f'Cluster 0: {y_4person[y_pred_kmeans == 0]}')
print(f'Cluster 1: {y_4person[y_pred_kmeans == 1]}')
print(f'Cluster 2: {y_4person[y_pred_kmeans == 2]}')
print(f'Cluster 3: {y_4person[y_pred_kmeans == 3]}')


Cluster 0: [ 2  2  2  2  2  2 37 37 37 37 37]
Cluster 1: [33 33 33 33 33 33 33 33 33 33]
Cluster 2: [17 17 17 17 17 17 17 17  2  2 37 37 37 37]
Cluster 3: [17 17  2  2 37]


- Does each cluster represent a true label?

  No, We can see the last cell. Cluster 1 and 2 approximately represent a true label, but cluster 0 and 3 represent a combination of labels and doesn't show a specific one.
- What is your conclusion?

  I think in our data, We can't recognize classes based on the similarity of their X. I mean, data have similar Xs with different labels. So the clustering method doesn't have a good performance on our data. The silhouette score shows this because if the clustering has a good performance it should be 1.

# Sec 5: Neural Net (Optional with Bonus Points)

Reshape the images so that they become of shape `64x64` **(P5-1: 5 points)**

In [ ]:
x_train_nn = x_train.reshape(-1, 64, 64)
x_val_nn = x_val.reshape(-1, 64, 64)
x_test_nn = x_test.reshape(-1, 64, 64)

Use different types of augmentation techniques (such as Random Flipping, Random Rotation, etc.) to increase the size of the training dataset. **(P5-2: 10 points)**

In [ ]:
augmented_x_train = x_train_nn.copy()
augmented_y_train = y_train.copy()
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
])

for i in range(3):
    augmented_x_train = np.concatenate((augmented_x_train, data_augmentation(x_train_nn)), axis=0)
    augmented_y_train = np.concatenate((augmented_y_train, y_train), axis=0)

Using the augmented training set, train a neural network consisting of 2D convolutional layers. The exact architecture of the network is arbitrary. However, the last layer should have the "softmax" function as its activation. Also, using "max pooling" layers after convolutional ones is advised.

Note: Don't get frustrated if your network does not appear to be a good one. Give it some time to be trained. For instance, you might need to train it for 100 epochs. Make sure to try that out before quitting!

**(P5-3: 20 points)**

In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(39, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

y_train_encoded = to_categorical(augmented_y_train, 39)
y_val_encoded = to_categorical(y_val, 39)
with tf.device('/GPU:0'):
  model.fit(augmented_x_train,y_train_encoded, epochs=100, batch_size=32, validation_data=(x_val_nn, y_val_encoded), callbacks=[checkpoint_callback],use_multiprocessing=True)


Epoch 1/100
36/40 [==========================>...] - ETA: 0s - loss: 3.6682 - accuracy: 0.0408
Epoch 1: val_accuracy improved from -inf to 0.05000, saving model to best_model.h5
40/40 [==============================] - 4s 21ms/step - loss: 3.6678 - accuracy: 0.0406 - val_loss: 3.6619 - val_accuracy: 0.0500
Epoch 2/100
38/40 [===========================>..] - ETA: 0s - loss: 3.6636 - accuracy: 0.0502
Epoch 2: val_accuracy did not improve from 0.05000
40/40 [==============================] - 0s 8ms/step - loss: 3.6637 - accuracy: 0.0500 - val_loss: 3.6601 - val_accuracy: 0.0500
Epoch 3/100
37/40 [==========================>...] - ETA: 0s - loss: 3.6610 - accuracy: 0.0498
Epoch 3: val_accuracy did not improve from 0.05000
40/40 [==============================] - 0s 12ms/step - loss: 3.6613 - accuracy: 0.0500 - val_loss: 3.6573 - val_accuracy: 0.0500
Epoch 4/100
39/40 [============================>.] - ETA: 0s - loss: 3.6599 - accuracy: 0.0513
Epoch 4: val_accuracy did not improve from 0.0

Measure the accuracy of the network on the validation and testing datasets **(P5-4: 5 points)**

In [ ]:
best_model = tf.keras.models.load_model('best_model.h5')

y_val_encoded = to_categorical(y_val, 39)
y_test_encoded = to_categorical(y_test, 39)
print(f'Validation accuracy: {best_model.evaluate(x_val_nn, y_val_encoded)[1]}')
print(f'Test accuracy: {best_model.evaluate(x_test_nn, y_test_encoded)[1]}')


2/2 [==============================] - 0s 8ms/step - loss: 0.2555 - accuracy: 0.9500
Validation accuracy: 0.949999988079071
2/2 [==============================] - 0s 7ms/step - loss: 0.4422 - accuracy: 0.9000
Test accuracy: 0.8999999761581421
